In [1]:
%matplotlib inline
import numpy as np                   # advanced math library

# for reproducibility
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
rseed = np.random.RandomState(0)

import matplotlib.pyplot as plt      # MATLAB like plotting routines
import random                        # for generating random numbers
from keras.models import Sequential  # Model type to be used

from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model
from keras.utils import np_utils                         # NumPy related tools

from keras.layers import Dense
from keras.models import Model
from keras.optimizers import adam_v2
from tensorflow.keras.callbacks import EarlyStopping


# import CNN tools
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetV2, InceptionV3, DenseNet201, ResNet50V2


from sklearn.model_selection import train_test_split # some helper from scikit for data split

#get rid of annoying GPU warnings (and others)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ImportError: cannot import name 'EfficientNetV2' from 'tensorflow.keras.applications' (/opt/conda/lib/python3.7/site-packages/keras/api/_v2/keras/applications/__init__.py)

In [ ]:
class_names = [
 "airplane",
 "automobile",
 "bird",
 "cat",
 "deer",
 "dog",
 "frog",
 "horse",
 "ship",
 "truck"    
]

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("------------------------------\n")
print(f"X_train's shape is {X_train.shape}")
print(f"y_train's shape is {y_train.shape}")
print(f"X_test's shape is {X_test.shape}")
print(f"y_test's shape is {y_test.shape}")
print(f"y_train has {np.max(y_train) + 1} classes")


In [ ]:
# Let's take a look at some of these images
fig, ax = plt.subplots(2, 5, figsize=(15, 6))
for i in range(2):
    for j in range(5):
        ind = (i * 5) + j
        ds_ex = np.where(y_train == ind)[0][0]        
        ax[i, j].imshow(X_train[ds_ex, ...])
        ax[i, j].set_title(class_names[ind])
plt.show()

In [ ]:
y_train = to_one_hot(y_train)
y_test = to_one_hot(y_test)

print(f"y_test's shape is {y_test.shape}")
print(f"y_train's shape is {y_train.shape}")

Aim:
- Split the training data into train:validation split of 80:20
- You are limited to a maximum of 20 Epochs!
- Train using training data but by evaluating the validation set
- Initialise a better model than ResNet50 for transfer learning.
- Maximize accuracy on the unseen test set
- Use transfer learning with appropriate frozen layers for faster training.
- Use batch normalization to improve training speed and accuracy.
- Use data augmentation to increase dataset diversity.
- Produce (four) line graphs of training and test losses and accuracies on a single plot!

In [ ]:
# Split the training data into train:validation split of 80:20
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=rseed)

print("New shapes\n")
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_val shape", X_val.shape)
print("y_val shape", y_val.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

In [ ]:
# we'll use this optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# we will also perform this in batch
batch_sz = 128
num_epochs = 12

model.fit(X_train, y_train, 
          batch_size=batch_sz, 
          epochs=num_epochs, 
          validation_data=(X_val, y_val), verbose=1)

In [ ]:

#This is the score for the model on the val data
score = model.evaluate(X_val, y_val)
print('Val score:', score[0])
print('Val accuracy:', score[1])
#This is the score for the model on the test data
score = model.evaluate(X_test, y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#this is my re-run before I fullyb Maximize the accuracy- I have modified the convolution layers and addes 
# some more attributes that I believe will help
# Initialize the model
modelv2 = Sequential()
# Convolution Layer 1
modelv2.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1), input_shape=(28, 28, 1)))
modelv2.add(Activation('relu'))
modelv2.add(BatchNormalization(axis=-1))

# Convolution Layer 2
modelv2.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1)))
modelv2.add(Activation('relu'))
modelv2.add(BatchNormalization(axis=-1))
modelv2.add(MaxPooling2D(pool_size=(2, 2))) 

# Convolution Layer 3
modelv2.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', strides=(1, 1)))
modelv2.add(Activation('relu'))
modelv2.add(BatchNormalization(axis=-1))
modelv2.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer to transition to fully connected layers


# Fully connected layers
modelv2.add(Dense(units=512, activation='relu'))
modelv2.add(Dropout(0.5))  # Adding dropout for regularization
modelv2.add(Flatten())
modelv2.add(Dense(units=10, activation='softmax'))  # Output layer

# Compile the model
modelv2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
modelv2.fit(X_train, y_train, 
          batch_size=batch_sz, 
          epochs=num_epochs, 
          validation_data=(X_val, y_val), verbose=1)
#This is the second score for the model on the val data
print('------------------------------------------------score----------------------------------------')
score = modelv2.evaluate(X_val, y_val)
print('Val score:', score[0])
print('Val accuracy:', score[1])

### this time around the accuracy was was much higher for both Test and Validation and now we can try to maximize

### Maximisation of Accuracy on the Test data 

In [ ]:
gen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image
        width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False
)  # randomly flip images

gen2 = ImageDataGenerator()

train_gen = gen.flow(X_train, y_train, batch_size=batch_sz)

# Fit the model
history = modelv2.fit(train_gen, batch_size=batch_sz, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

In [ ]:
# Evaluate the model on test data
test_score = modelv2.evaluate(X_test, y_test)

# Print test score and accuracy
print('Test score:', test_score[0])
print('Test accuracy:', test_score[1])

